# importando tabula e arquivos pdf
- importante que os PDFs estejam todos na mesma página dos arquivos

In [1]:
import tabula
import pandas as pd

lista_suap = tabula.read_pdf("suap/2023_04_RMA_suap.pdf", pages="1")
mes = '2023_04_RMA'
# lista em dataframe
df_suap = lista_suap[0]

# removendo linhas com valores nulos
df_suap = df_suap.dropna()
display(df_suap)

df_suap = df_suap[:-1]

# removendo e renomeando colunas
df_suap = df_suap.drop(columns=['Nome', 'Estoque', 'Entrada'])
df_suap['ED'] = df_suap['Cod.']
df_suap['saida'] = df_suap['Saída']
df_suap['saldo_suap'] = df_suap['Estoque.1']
df_suap = df_suap.drop(columns=['Cod.', 'Estoque.1', 'Saída'])

# substituindo caracteres para a conversão em numero
df_suap['saldo_suap'] = df_suap['saldo_suap'].str.replace(".", "", regex=True)
df_suap['saldo_suap'] = df_suap['saldo_suap'].str.replace(",", ".", regex=True)
df_suap['saida'] = df_suap['saida'].str.replace(".", "", regex=True)
df_suap['saida'] = df_suap['saida'].str.replace(",", ".", regex=True)


df_suap['saldo_suap'] = pd.to_numeric(df_suap['saldo_suap'])
df_suap['saida'] = pd.to_numeric(df_suap['saida'])
print("{:.2f}".format(df_suap['saldo_suap'].sum()))
print(df_suap.info())
display(df_suap)

Got stderr: May 02, 2023 3:29:08 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
May 02, 2023 3:29:08 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
May 02, 2023 3:29:12 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
May 02, 2023 3:29:12 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
May 02, 2023 3:29:12 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



,Cod.,Nome,Estoque,Entrada,Saída,Estoque.1
1,14,MATERIAL EDUCATIVO E ESPORTIVO,"31.843,07","0,00","0,00","31.843,07"
2,16,MATERIAL DE EXPEDIENTE,"55.397,81","0,00","38,24","55.359,57"
3,17,MATERIAL DE PROCESSAMENTO DE DADOS,"13.477,99","0,00","0,00","13.477,99"
4,19,MATERIAL DE ACONDICIONAMENTO E,"2.320,87","0,00","0,00","2.320,87"
6,21,MATERIAL DE COPA E COZINHA,"6.619,41","0,00","37,12","6.582,29"
7,22,MATERIAL DE LIMPEZA E PROD. DE,"15.661,38","0,00","518,00","15.143,38"
9,23,"UNIFORMES, TECIDOS E AVIAMENTOS","21.115,35","0,00","199,20","20.916,15"
10,24,MATERIAL P/ MANUT.DE BENS,"31.904,03","0,00","6,00","31.898,03"
12,25,MATERIAL P/ MANUTENCAO DE BENS,"199,60","0,00","0,00","199,60"
14,26,MATERIAL ELETRICO E ELETRONICO,"19.160,76","0,00","0,00","19.160,76"


229459.18
<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 1 to 20
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ED          15 non-null     object 
 1   saida       15 non-null     float64
 2   saldo_suap  15 non-null     float64
dtypes: float64(2), object(1)
memory usage: 480.0+ bytes
None


,ED,saida,saldo_suap
1,14,0.00,31843.07
2,16,38.24,55359.57
3,17,0.00,13477.99
4,19,0.00,2320.87
6,21,37.12,6582.29
7,22,518.00,15143.38
9,23,199.20,20916.15
10,24,6.00,31898.03
12,25,0.00,199.60
14,26,0.00,19160.76


In [2]:
# parâmetros lattice=True e guess=False para conseguir reconhecer e montar tabelas em arquivos com formatação diferente

lista_siafi = tabula.read_pdf("siafi/2023_04_RMA_siafi.pdf", guess=False, pages="1-2")

tabela1 = lista_siafi[0]
tabela1.columns = tabela1.iloc[7]
tabela1 = tabela1[8:20]

tabela2 = lista_siafi[1]
tabela2.columns = tabela2.iloc[7]
tabela2 = tabela2[8:15]

# Renomeando coluna por causa do caracter especial $

tabela1.columns = ['conta']
tabela2.columns = ['conta']

# unindo tabelas pela coluna conta
df_siafi = pd.merge(tabela1, tabela2, how = 'outer', on = 'conta')



# fatiando coluna para obter colunas ED e Saldo
df_siafi['ED'] = df_siafi['conta'].str[:4]
df_siafi['saldo_siafi'] = df_siafi['conta'].str[4:-1]

# removendo coluna ajustada
df_siafi = df_siafi.drop(columns=['conta'])

# utilize aqui para dropar as linhas extras quando some ou adiciona um novo ED
display(df_siafi)
df_siafi = df_siafi.drop(index=[15,16,17,18])

# substituindo caracteres para a conversão em numero
df_siafi['saldo_siafi'] = df_siafi['saldo_siafi'].str.replace(".", "", regex=True)
df_siafi['saldo_siafi'] = df_siafi['saldo_siafi'].str.replace(",", ".", regex=True)
df_siafi['ED'] = df_siafi['ED'].str.replace("P ", "", regex=True)
df_siafi['saldo_siafi'] = pd.to_numeric(df_siafi['saldo_siafi'])

print("{:.2f}".format(df_siafi['saldo_siafi'].sum()))
display(df_siafi)
print(df_siafi.info())

,ED,saldo_siafi
0,P 14,"31.843,07"
1,P 16,"55.397,81"
2,P 17,"13.477,99"
3,P 19,"2.320,87"
4,P 21,"6.619,41"
5,P 22,"15.661,38"
6,P 23,"21.115,35"
7,P 24,"31.904,03"
8,P 25,"199,60"
9,P 26,"19.160,76"


230285.72


,ED,saldo_siafi
0,14,31843.07
1,16,55397.81
2,17,13477.99
3,19,2320.87
4,21,6619.41
5,22,15661.38
6,23,21115.35
7,24,31904.03
8,25,199.60
9,26,19160.76


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ED           15 non-null     object 
 1   saldo_siafi  15 non-null     float64
dtypes: float64(1), object(1)
memory usage: 360.0+ bytes
None


In [6]:
# unindo data frames do suap e siafi em uma unica tabela
df_rma = pd.merge(df_suap, df_siafi, how = 'inner', on='ED')
df_rma = df_rma.reindex(columns=['ED','saldo_suap', 'saida', 'saldo_siafi'])

# calculando a diferença entre colunas 
df_rma['diferenca'] = df_rma['saldo_suap'] - df_rma['saldo_siafi']

# definindo a ação para cafa resultado
acao = []
for valor in df_rma['diferenca']:
    
    if valor > 0:
        baixa = "baixa no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "baixa no siafi"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

df_rma['acao'] = acao

display(df_rma)
df_rma.to_clipboard()
df_rma.to_excel(f'saida/{mes}.xlsx')

,ED,saldo_suap,saida,saldo_siafi,diferenca,acao
0,14,31843.07,0.00,31843.07,0.00,adequado
1,16,55359.57,38.24,55397.81,-38.24,baixa no siafi
2,17,13477.99,0.00,13477.99,0.00,adequado
3,19,2320.87,0.00,2320.87,0.00,adequado
4,21,6582.29,37.12,6619.41,-37.12,baixa no siafi
5,22,15143.38,518.00,15661.38,-518.00,baixa no siafi
6,23,20916.15,199.20,21115.35,-199.20,baixa no siafi
7,24,31898.03,6.00,31904.03,-6.00,baixa no siafi
8,25,199.60,0.00,199.60,0.00,adequado
9,26,19160.76,0.00,19160.76,0.00,adequado
